<a href="https://colab.research.google.com/github/sviteribuben/ML_01/blob/main/jun_ml_linear_regression_I_hw_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Урок 11. Домашняя работа

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома, где будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#necessary imports
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from numpy.linalg import inv
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML_01/3.10_non_linear.csv', sep=',')
data.head()

,x_train,y_train
0,0.138368,0.838812
1,0.157237,0.889313
2,0.188684,1.430040
3,0.685553,1.717309
4,0.874237,2.032588


In [6]:
# Функция принимает одномерный массив, а отдает n-мерный    
# Для каждой степени от 1 до pow возводит x в степень
def pow_array(source: list, pow: int):
    return np.array([source ** n for n in range(1, pow + 1)]).T

# обучаем на полученных данных предсказываем и отдаем скор
def r2_score_polynom(pow, data):
    X = pow_array(data.x_train, pow)
    model = LinearRegression().fit(X, data.y_train)
    y_pred = model.predict(X)
    error = r2_score(data.y_train, y_pred)
    return error

# Ищем лучший скоре и отдаем степень полинома
def best_polynom(data, max_n):
    errors = np.array([r2_score_polynom(n, data) for n in range(1,max_n+1)])    
    best_pow = np.argmax(errors) + 1
    return best_pow, errors[best_pow - 1] 

res = best_polynom(data, 10)
print(f'Подходящая степень полинома {res[0]}, ошибка {res[1]}')

Подходящая степень полинома 10, ошибка 0.9091133831302721


In [7]:
X = pow_array(data.x_train, 10)

model = LinearRegression().fit(X, data.y_train)
y_pred = model.predict(X)
y_pred

array([ 0.88210087,  1.01897613,  1.21593655,  1.90777396,  1.89320069,
        1.96799163,  1.99510718,  2.00255718,  2.05113546,  2.05904866,
        2.0713552 ,  2.08488909,  2.08789356,  2.05782449,  1.96152903,
        1.70752917,  1.25295923,  1.19221974,  1.03691947,  1.00291066,
        0.88643146,  0.87018974,  0.48418427,  0.44549173,  0.29608621,
        0.26126525,  0.20252307,  0.19922484,  0.17116334,  0.15216138,
        0.15216138,  0.1507041 ,  0.14577298,  0.13117666,  0.09362252,
        0.08594862,  0.0727596 ,  0.06757111, -0.00888383, -0.00456115,
        0.00499671,  0.0586598 ,  0.10034343,  0.22510023,  0.27123162,
        0.37545156,  0.56831668,  0.96442799,  0.9351922 ,  0.9351922 ])

**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисляемых коэффициентов. 

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [20]:
class CustomLinearReg:

    def init(self):
        self.w = None

    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1)
        self.w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

    def predict(self, X):
        Y_model = self.w[1][0]*X + self.w[0][0]
        return Y_model

reg = CustomLinearReg()
X = data.x_train.values.reshape(-1, 1)
y = data.y_train.values.reshape(-1, 1)
reg.fit(X,y)

y_pred = reg.predict(X)
print(r2_score(y_pred, y))

0.07495867693964542


In [8]:
class CustomLinearReg:

    def __init__(self):
        self.w = []
    
    def fit(self, X, y):
        X_new = [1 for i in range(1 + len(X) -1)] + X
        X_new = np.array(X_new).reshape(-1, 2, order='F')
        Y_new = np.array(y).reshape(-1, 1)
        X_T_X = (X_new.T).dot(X_new)
        X_T_X_inverted = inv(X_T_X)
        self.w = X_T_X_inverted.dot(X_new.T).dot(Y_new)
        return self
    
    def predict(self,X):
        X_new = [1 for i in range(1 + len(X) - 1)] + X
        X_new = np.array(X_new).reshape(-1, 2, order='F')
        return X_new.dot(self.w)

# Check
x_temp = [10, 60, 90, 110]
y_temp = [50, 62, 86, 151]

reg = CustomLinearReg().fit(x_temp, y_temp)

reg.predict(x_temp)

array([[ 36.46255507],
       [ 80.62555066],
       [107.12334802],
       [124.78854626]])

**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid) с помощью функции `train_test_split` из библиотеки `sklearn`
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [9]:
boston_dataset = load_boston()

boston_dataset.keys(), \
boston_dataset.feature_names, \
boston_dataset.target[:3], \
boston_dataset.data[:3]

(dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename']),
 array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
        'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7'),
 array([24. , 21.6, 34.7]),
 array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
         1.5300e+01, 3.9690e+02, 4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
         1.7800e+01, 3.9690e+02, 9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
         1.7800e+01, 3.9283e+02, 4.0300e+00]]))

In [10]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [11]:
boston.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

In [16]:
boston_dataset = load_boston()
X = boston_dataset.data
Y = boston_dataset.target

In [17]:
#splittint data 80% - train / 20% 
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In [15]:
# обучаем моедель
reg = LinearRegression().fit(X_train, y_train)
# предсказываем по валидационной выборке
y_pred = reg.predict(X_valid)
# получаем r2 score
r2_1 = r2_score(y_valid, y_pred)
y_train_2 = StandardScaler().fit_transform(y_train.reshape(-1,1)).reshape(-1) 
y_valid_2 = StandardScaler().fit_transform(y_valid.reshape(-1,1)).reshape(-1)
reg = LinearRegression().fit(X_train, y_train_2)
y_pred = reg.predict(X_valid)
r2_2 = r2_score(y_valid_2, y_pred)
print(f'R2 до преобразования {r2_1}, после - {r2_2}')

R2 до преобразования 0.6709339839115633, после - 0.5696363020639074


R2 score сократился. После Z преобразования ошибка не уменьшилась(

In [19]:
# another way
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2, random_state=42)

model_straight = LinearRegression().fit(X_train, y_train)
y_pred_straight = model_straight.predict(X_valid)
r2_score_straight = r2_score(y_valid, y_pred_straight)

print('r2_score_straight = %.8f' % r2_score_straight)

scaler = StandardScaler()
scaler_y = StandardScaler()

X_train_z, X_valid_z = scaler.fit_transform(X_train), scaler.transform(X_valid)
y_train_z, y_valid_z = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel(), scaler_y.transform(y_valid.reshape(-1, 1)).ravel()

model_z = LinearRegression().fit(X_train_z, y_train_z)

y_pred_normalized = model_z.predict(X_valid_z)

r2_score_normalized = r2_score(y_valid_z, y_pred_normalized)

print('r2_score_normalized = %.8f' % r2_score_normalized)


r2_score_straight = 0.66875949
r2_score_normalized = 0.66875949
